In [1]:
import chardet
import pandas as pd
import numpy as np
from datetime import date
today = date.today()
import mysql.connector

In [2]:
cnx = mysql.connector.connect(user='reporte', password='20rem07',
                              host='192.168.0.3',
                              database='remesa_remesa')

In [3]:
sql = '''SELECT 
  clie.CL_NOMFANTASIA,
  funcionario.FU_NOMBRE as EJECUTIVO,
  DATE_FORMAT(dr.DR_FECHA, '%d-%m-%Y') as DR_FECHA,
  deu.DD_RAZONSOCIAL,
  deu.DD_RUT,
  d.DC_ID,
  d.TD_ID,
  d.DC_NUMERO,
  dr.DR_ID,
  dil.DIL_NOMBRE,
  IF(d.CONCILIADO=0, "No", "Si") as DOC_PAGADO,
  REPLACE(d.DC_MONTO,".0000","") as MONTO,
  REPLACE(d.DC_SALDO,".0000","") as SALDO,
  deu.DD_DIRECCION,
  deu.DD_FONO,
  deu.DD_CONTACTO1,
  deu.DD_MAIL1,
  deu.DD_FONO1,
  deu.DD_CONTACTO2,
  deu.DD_FONO2,
  deu.DD_MAIL2,
  deu.DD_CONTACTO3,
  deu.DD_FONO3,
  deu.DD_MAIL3
  FROM documento d
  INNER JOIN cliente as clie ON clie.CL_RUT = d.CL_RUT
  INNER JOIN deudor as deu ON d.DD_RUT=deu.DD_RUT and d.CL_RUT=deu.CL_RUT
  INNER JOIN dilres_ultimas_dil dr ON d.DC_ID = dr.DC_ID 
  INNER JOIN diligencia as dil ON dil.DIL_ID = dr.ID_DIL
  INNER JOIN funcionario ON funcionario.FU_CODIGO =d.ID_USUARIO
  WHERE d.ELIMINADO = 0
  AND d.CL_RUT IN ('1-9','2-7','3-5','8-6','70-1','76.914.254-1')
  AND dr.DR_FECHA BETWEEN '2018-01-10 00:00:00' AND CURRENT_TIMESTAMP
  ORDER BY d.DC_ID ASC;
'''

In [4]:
df = pd.read_sql(sql, cnx, coerce_float=True)

In [5]:
df.SALDO = pd.to_numeric(df['SALDO']).astype(int)

In [6]:
df = df[df.DOC_PAGADO == 'No']

In [7]:
df.rename(columns={"DD_RUT": "Rut","CL_NOMFANTASIA":"Cartera","DIL_NOMBRE":"Diligencia","DR_FECHA":"Fecha Diligencia"}, 
          inplace=True)
df.loc[df['Cartera'] == 'REMESA 8 CAROL', 'Cartera'] = 'REMESA 8'
df.loc[df['Cartera'] == 'REMESA 8 IVAN', 'Cartera'] = 'REMESA 8'
df.loc[df['Cartera'] == 'INVERSIONES REMESA SPA', 'Cartera'] = 'REMESA 5'

In [8]:
df.Diligencia = df.Diligencia.str.upper()

In [9]:
df.loc[df["Diligencia"] == "COMPROMISO DE PAGO", "ESTADO"] = "CONVENIOS"
df.loc[df["Diligencia"] == "COMPROMISO DE PAGO INCUMPLIDO", "ESTADO"] = "CONVENIOS"
df.loc[df["Diligencia"] == "NORMALIZO", "ESTADO"] = "CONVENIOS"
df.loc[df["Diligencia"] == "COMPROMISO PAGO", "ESTADO"] = "CONVENIOS"
df.loc[df["Diligencia"] == "REQUERIMIENTO DE PAGO", "ESTADO"] = "CONVENIOS"

In [10]:
df.loc[df["Diligencia"] == "AVISO DE SUSPENCION", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] == "ENVIO DE CARTA RESULTADO POSITIVO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] == "ENVIO DE MAIL", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="ENVIO DE MAIL O INFORMACION A DEUDOR", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="BUSQUEDA DE TELEFONO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="BUZON DE VOZ", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="DEUDOR INUBICABLE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NUMERO NO CORRESPONDE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NUMERO NO EXISTE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="TELEFONO APAGADO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="TELEFONO NO CORRESPONDE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="TELÉFONO NO CORRESPONDE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="TELEFONO NO ENCONTRADO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="BUSQUEDA DE NUEVOS ANTECEDENTES", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="COPIA DOCUMENTO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="DESASIGNADO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="DEUDA CASTIGADA", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="FACTURA NO RECEPCIONADA", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="GERENCIA RETIENE EL PAGO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="LLAMADA PROGRAMADA", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="TELEFONO ENCONTRADO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NOTIFICACION PERSONAL", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NOTIFICACION POR CEDULA", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="VISITA A TERRENO EXITOSA", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="VERIFICACION EXITOSA", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="FUERA DE SERVICIO", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="FALTA DE DATOS", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NUMERO NO EXISTE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NUMERO NO CORRESPONDE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NÚMERO NO EXISTE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NÚMERO NO CORRESPONDE", "ESTADO"] = "SIN CONEXION"
df.loc[df["Diligencia"] =="NOTIFICACION  PERSONAL", "ESTADO"]="SIN CONEXION"
df.loc[df["Diligencia"] =="NOTIFICACION PERSONAL", "ESTADO"]="SIN CONEXION"
df.loc[df["Diligencia"] =="DOCUMENTO PARA ANÁLISIS", "ESTADO"]="SIN CONEXION"
df.loc[df["Diligencia"] =="DOCUMENTO PARA ANALISIS", "ESTADO"]="SIN CONEXION"
df.loc[df["Diligencia"] =="ENVIO DE SMS", "ESTADO"]="SIN CONEXION"
df.loc[df["Diligencia"] =="BUSQUEDAS POSITIVAS", "ESTADO"]="SIN CONEXION"
df.loc[df["Diligencia"] =="VISITA A TERRENO FALLIDA", "ESTADO"]="SIN CONEXION"

In [11]:
df.loc[df["Diligencia"]=="VERIFICACION FALLIDA","ESTADO"]="NO CONTESTA"
df.loc[df["Diligencia"]=="DEUDOR NO CONTESTA","ESTADO"]="NO CONTESTA"
df.loc[df["Diligencia"]=="NO CONTESTA","ESTADO"]="NO CONTESTA"
df.loc[df["Diligencia"]=="REITERADOS LLAMADOS, NO CONTESTA","ESTADO"]="NO CONTESTA"
df.loc[df["Diligencia"]=="PRIMERA GESTION","ESTADO"]="NO CONTESTA"

In [12]:
df.loc[df['Diligencia']=="NO PAGARÁ / SIN VOLUNTAD","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="INDICA QUE ABONO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR INSOLVENTE","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR INDICA QUE CANCELO DEUDA TOTAL","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CON INTENCION DE PAGO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEJA RECADO CON TERCERO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR FALLECIDO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="RECADO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CONFIRMA PAGADO EN OFICINAS DEL CLIENTE","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DESCONOCE DEUDA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR ARGUMENTA SITUACION ESPECIAL","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR EN QUIEBRA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR CORTA LA LLAMADA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DEUDOR INDICA QUE CANCELO DEUDA AL DIA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="ENVIARA COMPROBANTE DE PAGO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="NIEGA DEUDA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="NO PAGARA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="SIN VOLUNTAD","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="RECLAMO POR NORMALIZAR","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="RESPUESTA DEL CLIENTE","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="SIN INTENCION DE PAGO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="SIN VOLUNTAD DE PAGO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="SOLICITA VOLVER A LLAMAR","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="SOLICITUD ESTADO DE CUENTA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="YA PAGO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CONFIRMADO PAGADO EN TRANSFERENCIA O DEPOSITO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="RECONOCE REPACTACION","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CONFIRMA PAGADO EN OFICINAS DEL CLIENTE","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CONFIRMADO PAGADO EN TRANSFERENCIA O DEPOSITO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="FACTURA NO RECEPCIONADA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="GERENCIA RETIENE EL PAGO","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="RECONOCE REPACTACION","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CONFIRMA PAGADO EN SERVIPAG","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="SOLICITUD DE REFACTURACION","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="CANCELA CON CHEQUE A FECHA","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="REVISION DOCUMENTO PAGADO CON COMPROBANTE","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="FACTURA EN REVISION","ESTADO"]="CONTESTA"
df.loc[df['Diligencia']=="DOCUMENTO CONFIRMADO CANCELADO","ESTADO"]="CONTESTA"

In [13]:
df.head()

,Cartera,EJECUTIVO,Fecha Diligencia,DD_RAZONSOCIAL,Rut,DC_ID,TD_ID,DC_NUMERO,DR_ID,Diligencia,...,DD_CONTACTO1,DD_MAIL1,DD_FONO1,DD_CONTACTO2,DD_FONO2,DD_MAIL2,DD_CONTACTO3,DD_FONO3,DD_MAIL3,ESTADO
0,REMESA 5,EMPEX HMCIA,20-04-2022,CALLET CONTRERAS FILOMENA ANA PATRICIA,10.003.170-1,1874052,44,8445065-06,23945612,SOLICITA VOLVER A LLAMAR,...,,joel.espinoza2717@gmail.com,,,,,,,joel.espinoza2717@gmail.com,CONTESTA
1,REMESA 5,EMPEX HMCIA,20-04-2022,CALLET CONTRERAS FILOMENA ANA PATRICIA,10.003.170-1,1874053,44,8445065-01,23945611,SOLICITA VOLVER A LLAMAR,...,,joel.espinoza2717@gmail.com,,,,,,,joel.espinoza2717@gmail.com,CONTESTA
2,REMESA 5,EMPEX HMCIA,20-04-2022,CALLET CONTRERAS FILOMENA ANA PATRICIA,10.003.170-1,1874054,44,8445065-02,23945610,SOLICITA VOLVER A LLAMAR,...,,joel.espinoza2717@gmail.com,,,,,,,joel.espinoza2717@gmail.com,CONTESTA
3,REMESA 5,EMPEX HMCIA,20-04-2022,CALLET CONTRERAS FILOMENA ANA PATRICIA,10.003.170-1,1874055,44,8445065-03,23945609,SOLICITA VOLVER A LLAMAR,...,,joel.espinoza2717@gmail.com,,,,,,,joel.espinoza2717@gmail.com,CONTESTA
4,REMESA 5,EMPEX HMCIA,20-04-2022,CALLET CONTRERAS FILOMENA ANA PATRICIA,10.003.170-1,1874056,44,8445065-04,23945608,SOLICITA VOLVER A LLAMAR,...,,joel.espinoza2717@gmail.com,,,,,,,joel.espinoza2717@gmail.com,CONTESTA


In [14]:
dil_conv = df[df.ESTADO == "CONVENIOS"]
dil_nocont = df[df.ESTADO == "NO CONTESTA"]
dil_sin = df[df.ESTADO == "SIN CONEXION"]
dil_con = df[df.ESTADO == "CONTESTA"]

In [15]:
print('BH')
print('Hola Alberto, como estas?\n')
print(f"Reporte del dia: {today}\n")
print(f'Cantidad de Rut: {df.Rut.drop_duplicates().count()}')
print(f'Saldo Total al dia ${df.SALDO.sum()}\n')
print(f'Convenios RUT ND: {dil_conv.Rut.drop_duplicates().count()}')
print(f'Saldo Convenios ${dil_conv.SALDO.sum()}\n')
print(f'Contesta RUT ND: {dil_con.Rut.drop_duplicates().count()}')
print(f'Saldo Contesta ${dil_con.SALDO.sum()}\n')
print(f'Suma RUT ND Convenios+Contesta: {dil_conv.Rut.drop_duplicates().count()+dil_con.Rut.drop_duplicates().count()}')
print(f'Suma Saldo Convenios+Contesta ${dil_conv.SALDO.sum()+dil_con.SALDO.sum()}\n')
print(f'No Contesta RUT ND: {dil_nocont.Rut.drop_duplicates().count()}')
print(f'Saldo No Contesta ${dil_nocont.SALDO.sum()}\n')
ccr = dil_conv.Rut.drop_duplicates().count()+dil_con.Rut.drop_duplicates().count()
print(f'Suma Rut Conv, Cont, No Cont: {ccr + dil_nocont.Rut.drop_duplicates().count()}')
ccs = dil_conv.SALDO.sum()+dil_con.SALDO.sum()
print(f'Suma Saldo Conv, Cont, No Cont: ${ccs + dil_nocont.SALDO.sum()}\n')
print(f'RUT ND Sin Conexion: {dil_sin.Rut.drop_duplicates().count()}')
print(f'Saldo Sin Conexion ${dil_sin.SALDO.sum()}')

BH
Hola Alberto, como estas?

Reporte del dia: 2022-08-05

Cantidad de Rut: 30883
Saldo Total al dia $31886025220

Convenios RUT ND: 4465
Saldo Convenios $4856744344

Contesta RUT ND: 8139
Saldo Contesta $8530478096

Suma RUT ND Convenios+Contesta: 12604
Suma Saldo Convenios+Contesta $13387222440

No Contesta RUT ND: 10347
Saldo No Contesta $10755975702

Suma Rut Conv, Cont, No Cont: 22951
Suma Saldo Conv, Cont, No Cont: $24143198142

RUT ND Sin Conexion: 8220
Saldo Sin Conexion $7742827078


In [16]:
df.to_excel(f"{today}.xlsx")

In [24]:
df[df.EJECUTIVO=='EMPEX HMCIA'].DC_NUMERO.count(), df[df.EJECUTIVO=='UNICOB'].DC_NUMERO.count()

(9535, 28244)

In [30]:
df.groupby(by=['EJECUTIVO','ESTADO','SALDO']).sum()

DC_ID  TD_ID      DR_ID
EJECUTIVO      ESTADO    SALDO                             
Adrian Perdomo CONTESTA  4        2486598     44   24582034
                         250      2589091     55   24444488
                         5632     2487185     44   24590710
                         7557     2479285     44   24586100
                         11220    2481125     44   23547457
...                                   ...    ...        ...
equipo2        CONVENIOS 86769    2211882     58   22888069
                         89406    1969058     58   22954119
                         107539  15551772    352  183633011
                         107540   1943976     44   22954120
                         141666   5779536    132   68664213

[66315 rows x 3 columns]